[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alexwolson/carte-biozone-workshop/blob/main/Lab-1-1.ipynb)

# CARTE-BioZone Workshop on Machine Learning
#### Monday, August 28, 2023
#### Lab 1, Day 1: Introduction
##### Lab author: Kyle E.C. Booth, edited by Alex Olson

#### Introduction

In this lab, we will be using the popular machine learning library [scikit-learn](https://scikit-learn.org/stable/) in tandem with a popular scientific computing library in Python, [NumPy](https://www.numpy.org/), to investigate basic machine learning principles and models. The topics that will be covered in this lab include:
* Introduction to scikit-learn and NumPy
* Exploratory data analysis (EDA)
* Nearest neighbors classification algorithm
* Nested cross-validation

*Note:* Some other useful Python libraries include [matplotlib](https://matplotlib.org/) (for plotting/graphing) and [Pandas](https://pandas.pydata.org/) (for data analysis), though we won't be going into detail on these in this lab. 

##### Notebooks
This lab will be using notebooks as a Python development environment. Hopefully you're somewhat familiar with them. Write your code in *cells* (this is a cell!) and execute your code by pressing the *play* button (up top) or by entering *shift+enter*. To format a cell for text, you can select "Markdown" from the dropdown - the default formatting is "Code", which will usually be what you want. 

#### Getting started
Let's get started. First, we're going to test that we're able to import the required libraries.  
**>> Run the code in the next cells** to import scikit-learn and NumPy.

In [ ]:
# Check if we are running on Google Colab, or locally
import sys

IN_COLAB = "google.colab" in sys.modules

In [ ]:
if not IN_COLAB:  # Colab already has these installed
    !pip install -q numpy scikit-learn matplotlib seaborn

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score

### NumPy Basics

Great. Let's move on to our next topic: getting a handle on NumPy basics. You can think of NumPy as sort of like a MATLAB for Python (if that helps). The main object is multidimensional arrays, and these come in particularly handy when working with data and machine learning algorithms.

Let's create a 2x4 array containing the numbers 1 through 8 and conduct some basic operations on it.  
**>> Run the code in the next cell to create and print the array.***

In [ ]:
array_2x4 = np.arange(8).reshape(2, 4)
array_2x4

We can access the shape, number of dimensions, data type, and number of elements in our array as follows:  
*(Tip: use "print()" when you want a cell to output more than one thing, or you want to append text to your output, otherwise the cell will output the last object you call, as in the cell above)*

In [ ]:
# f-strings are a way to embed expressions inside string literals, using curly braces {}.
# The expressions inside the braces are evaluated at runtime and then formatted using the format string syntax.
print(f"Shape: {array_2x4.shape}")
print(f"Dimensions: {array_2x4.ndim}")
print(f"Data type: {array_2x4.dtype.name}")
print(f"Number of elements: {array_2x4.size}")

If we have a Python list containing a set of numbers, we can use it to create an array:

In [ ]:
# You can easily create a NumPy array from a regular Python list. This can be useful when working with sequences of numerical data.
# Here, we will create a NumPy array from a list containing the first nine numbers of the Fibonacci sequence.

fibonacci_list = [0, 1, 1, 2, 3, 5, 8, 13, 21]
fibonacci_array = np.array(fibonacci_list)
fibonacci_array

And we can do it for nested lists as well, creating multidimensional NumPy arrays:

In [ ]:
two_d_list = [[1, 2, 3], [4, 5, 6]]
two_d_array = np.array(two_d_list)
two_d_array

We can also index and slice NumPy arrays like we would do with a Python list or another container object as follows:

In [ ]:
array_one_dim = np.arange(10)
print(f"Originally: {array_one_dim}")
print(f"First four elements: {array_one_dim[:4]}")
print(f"After the first four elements: {array_one_dim[4:]}")
print(f"The last element: {array_one_dim[-1]}")

And we can index/slice multidimensional arrays, too.

In [ ]:
# The syntax `array[:,0]` for example, accesses the first element of each row (i.e., the first column).

multi_array = np.array([[1, 2, 3], [4, 5, 6]])
print(f"Originally:\n{multi_array}")
print(f"First row only: {multi_array[0]}")
print(f"First column only: {multi_array[:, 0]}")

#### Sneak preview

Often, when designing a machine learning classifier, it can be useful to compare an array of predictions (0 or 1 values) to another array of true values. We can do this pretty easily in NumPy to compute the *accuracy* (e.g., the number of values that are the same), for example, as follows:

In [ ]:
# We are comparing two arrays to find the 'accuracy', which is the proportion of matching elements.
# In a machine learning context, this might represent the percentage of correct predictions.

true_values = [0, 0, 1, 1, 1, 1, 1, 0, 1, 0]
predictions = [0, 0, 0, 1, 1, 1, 0, 1, 1, 0]

true_values_array = np.array(true_values)
predictions_array = np.array(predictions)

# Using NumPy's mean function to compute accuracy
accuracy = np.mean(true_values_array == predictions_array)
print(f"Accuracy: {accuracy * 100}%")

In the previous cell, we took two Python lists, converted them to NumPy arrays, and then used a combination of np.sum() and .size to compute the accuracy (proportion of elements that are pairwise equal). A tiny bit more advanced, but demonstrates the power of NumPy arrays.

### Scikit-learn Basics

Scikit-learn is a great library to use for doing machine learning in Python. Data preparation, exploratory data analysis (EDA), classification, regression, clustering; it has it all. 

Scikit-learn usually expects data to be in the form of a 2D matrix with dimensions *n_samples x n_features* with an additional column for the target. To get acquainted with scikit-learn, we are going to use the [iris dataset](https://archive.ics.uci.edu/ml/datasets/iris), one of the most famous datasets in pattern recognition. 

Each entry in the dataset represents an iris plant, and is categorized as: 

* Setosa (class 0)
* Versicolor (class 1)
* Virginica (class 2)

These represent the target classes to predict. Each entry also includes a set of features, namely:

* Sepal width (cm)
* Sepal length (cm)
* Petal length (cm)
* Petal width (cm)

In the context of machine learning classification, the remainder of the lab is going to investigate the following question:  
*Can we design a model that, based on the iris sample features, can accurately predict the iris sample class? *

Scikit-learn has a copy of the iris dataset readily importable for us. Let's grab it now and conduct some EDA.

In [ ]:
iris_data = load_iris()
feature_data = iris_data.data

**YOUR TURN:** "feature_data" now contains the feature data for all of the iris samples. 
* What is the shape of this feature data? ________________
* The data type? ________________
* How many samples are there? ________________
* How many features are there? ________________

In [ ]:
## Enter your code here

Next, we will save the target classification data in a similar fashion.

In [ ]:
target_data = iris_data.target
target_names = iris_data.target_names

**YOUR TURN:**
* What values are in "target_data"? ________________
* What is the data type? ________________
* What values are in "target_names"? ________________
* What is the data type? ____________
* How many samples are of type "setosa"? ________________

In [ ]:
## Enter your code here

We can also do some more visual EDA by plotting the samples according to a subset of the features and coloring the data points to coincide with the sample classification. We will use [matplotlib](https://matplotlib.org/), a powerful plotting library within Python, to accomplish this.

For example, let's plot sepal width vs. sepal length.


In [ ]:
# Preparing data for Seaborn plotting
setosa = feature_data[target_data == 0]
versicolor = feature_data[target_data == 1]
virginica = feature_data[target_data == 2]

# Plotting setosa
sns.scatterplot(x=setosa[:, 0], y=setosa[:, 1], label="setosa")

# Plotting versicolor
sns.scatterplot(x=versicolor[:, 0], y=versicolor[:, 1], label="versicolor")

# Plotting virginica
sns.scatterplot(x=virginica[:, 0], y=virginica[:, 1], label="virginica")

# Adding labels and title
plt.xlabel("sepal length (cm)")
plt.ylabel("sepal width (cm)")
plt.title("Visual EDA")
plt.legend(title="Class")
plt.show()  # Sometimes this explicit call is optional - depends on your environment

In the above step, we used boolean indexing to filter the feature data based on the target data class. This allowed us to create a scatter plot for each of the iris classes and distinguish them by color.

*Observations*: We can see that the "setosa" class typically consists of medium-to-high sepal width with low-to-medium sepal length, while the other two classes have lower width and higher length. The "virginica" class appears to have the largest combination of the two. 

**YOUR TURN:** 
* Which of the iris classes is seperable based on sepal characteristics? ________________
* Which of the iris classes is not? ________________
* Can we (easily) visualize each of the samples w.r.t. all features on the same plot? Why/why not? ________________

### Creating a k-Nearest Neighbors Classifier

Now that we've explored the data a little bit, we're going to use scikit-learn to create a k-nearest neighbors classifier for the data. Effectively we'll be developing a model whose job it is to build a relationship over input feature data (sepal and petal characteristics) that predicts the iris sample class (e.g. "setosa"). This is an example of a *supervised learning* task; we have all the features and all the target classes.

Nearest neightbors classifiers are quite simple. They predict the class of a new data sample based off the *nearest* data points to that sample. The 'nearest' metric is calculated via a distance function (often [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance)). 

<img src="https://github.com/lyeskhalil/mlbootcamp/blob/master/img/knn.png?raw=1" alt="knn" width="200"/>

For example, in the above diagram, suppose we are looking to classify the green circle as either a red triangle, or a blue square. If k = 1 (i.e., we look at one neighbor), our model would predict *red triangle*. If k=2, it would still predict *red triangle*. If k=3, the model would predict *red triangle* as it is the *majority* class of the 3 nearest neighbors. It isn't until k=5 that the algorithm actually predicts *blue square*.

Model creation in scikit-learn follows a **data prep -> fit -> predict** process. The "fit" function is where the actual model is trained and parameter values are selected, while the "predict" function actually takes the trained model and applies it to the new samples.

First, we're going to save our feature data into an array called 'X' and our target data into an array called 'y'. We don't *need* to do this, but it is traditional to think of the problem using this notation.

In [ ]:
X = feature_data
y = target_data

Next, we create our nearest neighbor classifier object:

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)

And then we *fit* it to the data (i.e., train the classifier).

In [ ]:
knn.fit(X, y)

Now we have a model! If you're new to this, you've officially built your first machine learning model. If you use "knn.predict(*[[feature array here]]*)", you can use your trained model to predict the class of a new iris sample. 

**YOUR TURN:**
* What is the predicted class of a new iris sample with feature vector [3,4,5,2]? What is its name? ________________
* Do you think this model is overfit or underfit to the iris dataset? Why? ________________
* How many neighbors does our model consider when classifying a new sample? ________________

In [ ]:
# Insert code here

As you may have noted in the previous cell, we've trained this classifier on our *entire dataset*. This typically isn't done in practice and results in overfitting to the data. Here's a bit of a tricky question:

**YOUR TURN:**
* If we use our classifier to predict the classes of the iris samples that were used to train the model itself, what will our overall accuracy be? ________________

We can validate our hypothesis fairly easily using either: i) the NumPy technique for calculating accuracy we used earlier in the lab, or ii) scikit-learn's in-house "accuracy_score()" function.

Let's use our technique first:

In [ ]:
accuracy = np.mean(target_data == knn.predict(feature_data))
print(f"Accuracy: {accuracy * 100}%")

and then using scikit-learn's customized function:

In [ ]:
accuracy = accuracy_score(target_data, knn.predict(feature_data))
print(f"Accuracy: {accuracy * 100}%")

We see that our classifier has achieved 100% accuracy (and both calculation methods agree)!

**DISCUSSION:** 
* Why do you think the model was able to achieve such a "great" result? ______________________
* What does this really tell us?  __________________________________
* Do you expect the model to perform this well on new data? __________________________________

### Cross Validation

A popular way to mitigate this overfitting issue is to train your model on *some* of the data (the training set) and validate your model on the remaining data (the validation set). You will then select the model/configuration that performs best on the validation data. The train/validate division of the data is usually done with a 70%/30% split. Often, practitioners will use a third data set, the test set (or hold-out set), to get a sense for how their best model performs on unseen, real-world data. In this scenario, you will tune your models to perform best on the validation set and then test their "real-world" performance on the unseen test set.

Sometimes applications don't have enough data to do these splits meaningfully (e.g., the test data is only a few samples). In these cases, *cross-validation* is a useful technique (and, indeed, has become standard in machine learning practice). 

The general premise of "k-folds" cross validation is to first divide the entire dataset (grey) into a training set (green) and a test set (unseen data, blue). Then, we divide the training set into different folds and use these folds to form new sub-training and sub-test sets. We select the model configuration that performs the best on all of these. The below figure provides a nice visualization for what's going on here:

<img src="https://github.com/lyeskhalil/mlbootcamp/blob/master/img/cross-val.png?raw=1" alt="cross-val" width="500"/>


Accomplishing k-folds cross validation in scikit-learn is a manageable task. First, we divide our data into a train and test set, then we conduct the cross validation and look at the mean scores across the splits, then we conduct our final evaluation.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    feature_data, target_data, test_size=0.3, random_state=0
)

We have divided our data into two sections: training data (70% of the data) and testing data (30% of the data). Now we will fit our nearest neighbors classifier to the training data with 5 folds and see how it performs.

In [ ]:
scores = cross_val_score(knn, X_train, y_train, cv=5)
print(f"Accuracy: {scores.mean():.2f} (+/- {scores.std() * 1.96:.2f})")

Our cross-validated model has an accuracy of 94% across all the splits on the training data. If we think that is a reasonable value, we can train our final model on the training data and then see how it performs on the held-out test data. 

##### Comparing classifiers
However, to get a true sense for the utility of cross-validation, let's create a second nearest neighbors classifier that uses two neighbors instead of one. 

In [ ]:
knn_2_neighbors = KNeighborsClassifier(n_neighbors=2)
scores = cross_val_score(knn_2_neighbors, X_train, y_train, cv=5)
print(f"Accuracy: {scores.mean():.2f} (+/- {scores.std() * 1.96:.2f})")

As we see above, our second classifier (the one with two neighbors) actually performs worse when cross-validated (92% vs. 94% mean accuracy on the 5 folds)! So, we'll stick with the first one.

Evaluating the model on held-out test data is a critical step in assessing its performance. By using data that the model has not seen during training, we can gauge how well the model generalizes to new, unseen data. This provides a more realistic estimation of how the model would perform in a real-world scenario where it encounters data points it has not seen before.

Let's train the chosen model on the training data and use it to predict the final held-out test data. Evaluating on the test data will help us understand the model's ability to generalize, which is a key aspect of a reliable and robust machine learning model.

In [ ]:
knn.fit(X_train, y_train)
accuracy = accuracy_score(y_test, knn.predict(X_test))
print(f"Test set accuracy: {accuracy * 100:.2f}%")

And we see our model has a 97.8% accuracy on the held out test data (30% of the original dataset).

## Bonus Task: Confusion Matrix Analysis

A confusion matrix is a table used to describe the performance of a classification model on a set of data for which the true values are known. It is especially powerful for multi-class classification, like our iris dataset.

In this bonus task, you will create a confusion matrix for the k-nearest neighbors classifier you trained earlier on the test data. Then, calculate specific metrics like precision, recall, and F1-score for each class. Here's a step-by-step guide to help you get started:

1. Predict the Test Data: You've already done this, so you can simply use the predictions you made for the test data. 
2. Create the Confusion Matrix: You can use scikit-learn's confusion_matrix function for this.
3. Analyze the Matrix: Calculate precision, recall, and F1-score for each class. You might find scikit-learn's classification_report helpful or calculate them manually.

Below is a snippet to help you create the confusion matrix. The rest is up to you!

In [ ]:
from sklearn.metrics import confusion_matrix

# Predict the test data
y_pred = knn.predict(X_test)

# Create the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Now it's your turn to analyze this matrix and calculate precision, recall, and F1-score for each class!